In [59]:
import pandas as pd
import numpy as np
import glob, os
import nltk
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

In [60]:
path = '/content/drive/MyDrive/ml-data/Document-clustering'
all_files = glob.glob(os.path.join(path, "*.data"))
filename_list = []
opinion_text = []

for file_ in all_files:
  df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
  filename_ = file_.split('/')[-1]
  filename = filename_.split('.')[0]
  filename_list.append(filename)
  opinion_text.append(df.to_string())

document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,accuracy_garmin_nuvi_255W_gps,...
1,bathroom_bestwestern_hotel_sfo,...
2,battery-life_amazon_kindle,...
3,comfort_honda_accord_2008,...
4,buttons_amazon_kindle,...


In [61]:
import string

def LemTokens(tokens):
  lemmer = nltk.stem.WordNetLemmatizer()
  return [lemmer.lemmatize(token) for token in tokens]

def LemNormalize(text):
  remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
  return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english', \
                             ngram_range=(1,2), min_df=0.05, max_df=0.85)
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [70]:
print(tfidf_vect.get_feature_names())
print(feature_vect.toarray().shape)
print(feature_vect)

['0 5', '0 great', '0 room', '00', '05', '06', '07', '08', '1 2', '1 block', '1 great', '10 did', '10 great', '10 inch', '10 min', '10 minute', '10 room', '100', '100 accurate', '1005ha', '101', '102', '103', '104', '105', '105 room', '106', '107', '108', '109', '11 screen', '110', '111', '112', '113', '113 room', '114', '115', '116', '117', '118', '119', '119 hotel', '12 great', '120', '121', '121 hotel', '122', '123', '124', '125', '125 room', '126', '126 staff', '127', '128', '129', '129 room', '130', '131', '131 room', '132', '133', '134', '134 room', '135', '136', '137', '138', '139', '14 ha', '140', '141', '141 great', '142', '143', '143 room', '144', '145', '146', '147', '147 room', '148', '149', '15 minute', '15 second', '150', '151', '152', '153', '154', '154 room', '155', '156', '157', '158', '159', '159 hotel', '160', '161', '162', '162 wa', '163', '164', '165', '166', '167', '168', '169', '17 room', '170', '171', '172', '173', '174', '175', '176', '176 wa', '177', '178', '1

In [64]:
from sklearn.cluster import KMeans

km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=1)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [65]:
document_df['cluster_label'] = cluster_label
document_df.head(5)

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,1
1,bathroom_bestwestern_hotel_sfo,...,2
2,battery-life_amazon_kindle,...,1
3,comfort_honda_accord_2008,...,0
4,buttons_amazon_kindle,...,1


In [66]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
3,comfort_honda_accord_2008,...,0
5,comfort_toyota_camry_2007,...,0
21,gas_mileage_toyota_camry_2007,...,0
20,interior_honda_accord_2008,...,0
15,interior_toyota_camry_2007,...,0
16,mileage_honda_accord_2008,...,0
42,performance_honda_accord_2008,...,0
25,quality_toyota_camry_2007,...,0
46,seats_honda_accord_2008,...,0
33,transmission_toyota_camry_2007,...,0


In [67]:
document_df[document_df['cluster_label']==1].sort_values(by='filename') # KINDLE

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,1
2,battery-life_amazon_kindle,...,1
6,battery-life_ipod_nano_8gb,...,1
8,battery-life_netbook_1005ha,...,1
4,buttons_amazon_kindle,...,1
18,directions_garmin_nuvi_255W_gps,...,1
17,display_garmin_nuvi_255W_gps,...,1
7,eyesight-issues_amazon_kindle,...,1
9,features_windows7,...,1
14,fonts_amazon_kindle,...,1


In [68]:
document_df[document_df['cluster_label']==2].sort_values(by='filename') # HOTEL

,filename,opinion_text,cluster_label
1,bathroom_bestwestern_hotel_sfo,...,2
22,food_holiday_inn_london,...,2
12,food_swissotel_chicago,...,2
23,free_bestwestern_hotel_sfo,...,2
19,location_bestwestern_hotel_sfo,...,2
11,location_holiday_inn_london,...,2
24,parking_bestwestern_hotel_sfo,...,2
26,price_holiday_inn_london,...,2
28,room_holiday_inn_london,...,2
31,rooms_bestwestern_hotel_sfo,...,2


In [69]:
cluster_centers = km_cluster.cluster_centers_
print(cluster_centers.shape)
print(cluster_centers)

(3, 4611)
[[0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.01005322 0.         0.         ... 0.00706287 0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]]


####get Core words

In [80]:
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
  cluster_details = {}
  centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]

  for cluster_num in range(clusters_num):
    cluster_details[cluster_num] = {}
    cluster_details[cluster_num]['cluster'] = cluster_num
    top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
    top_feature = [ feature_names[ind] for ind in top_feature_indexes ]
    top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()

    cluster_details[cluster_num]['top_features'] = top_feature
    cluster_details[cluster_num]['top_features_value'] = top_feature_values
    filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
    filenames = filenames.values.tolist()

    cluster_details[cluster_num]["filenames"] = filenames
  return cluster_details     

    

In [83]:
def print_cluster_details(cluster_details):
  for cluster_num, cluster_detail in cluster_details.items():
    print("#### Cluster {0}".format(cluster_num))
    print("Top Features : ", cluster_detail['top_features'])
    print("Review files : ", cluster_detail['filenames'][:7])
    print("===========")

In [85]:
feature_names = tfidf_vect.get_feature_names()
print(feature_names)
cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df, feature_names=feature_names, clusters_num=3, top_n_features=10)
print_cluster_details(cluster_details)

['0 5', '0 great', '0 room', '00', '05', '06', '07', '08', '1 2', '1 block', '1 great', '10 did', '10 great', '10 inch', '10 min', '10 minute', '10 room', '100', '100 accurate', '1005ha', '101', '102', '103', '104', '105', '105 room', '106', '107', '108', '109', '11 screen', '110', '111', '112', '113', '113 room', '114', '115', '116', '117', '118', '119', '119 hotel', '12 great', '120', '121', '121 hotel', '122', '123', '124', '125', '125 room', '126', '126 staff', '127', '128', '129', '129 room', '130', '131', '131 room', '132', '133', '134', '134 room', '135', '136', '137', '138', '139', '14 ha', '140', '141', '141 great', '142', '143', '143 room', '144', '145', '146', '147', '147 room', '148', '149', '15 minute', '15 second', '150', '151', '152', '153', '154', '154 room', '155', '156', '157', '158', '159', '159 hotel', '160', '161', '162', '162 wa', '163', '164', '165', '166', '167', '168', '169', '17 room', '170', '171', '172', '173', '174', '175', '176', '176 wa', '177', '178', '1